<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ed85ed3c8eecc7502eb3bb2a60b8ed9865f04d83b43802c11707c472cc389e6f
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-15 09:26:57
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.18 L
Current:  1.42 C
-------------------
Today PnL: 8.53 K (0.06%)
Current PnL: -17.52 L (-11.78%)
CY Booked + Current PnL: -6.49 L (-4.37%)
-------------------
Total profit:  2.06 L
Total loss:  -19.58 L
-------------------
Total Booked + Current PnL: 20.49 L (16.84%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.78%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.62 L (59.02%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.45%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.53,59.0,H-LC,11.14,172887.0,16710.0,9388.0,-0.20,10.70,5.43,16.71,22.0,1.78,1.26,30.00,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-11.34,63.0,M-LC,4.26,218956.0,17476.0,9744.0,0.33,8.67,4.45,13.51,60.0,1.79,1.59,22.36,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,102.77,55.0,M-SC,4.73,87377.0,-13400.0,13491.0,-0.45,-13.30,15.44,0.09,245.0,-0.99,0.64,14.79,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-42.00,74.0,H-MC,5.11,116330.0,15826.0,14809.0,0.55,15.75,12.73,30.49,79.0,1.07,0.85,53.25,MH,ATH,METALS
41,ITC,452.00,-37.50,55.0,H-LC,2.90,202446.0,2308.0,18585.0,0.06,1.15,9.18,10.44,4.0,0.12,1.47,6.78,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-21.22,41.0,H-MC,2.20,135589.0,-199.0,42955.0,-0.14,-0.15,31.68,31.48,107.0,-0.00,0.99,11.95,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.51,59.0,H-MC,1.76,246792.0,-939.0,75987.0,0.60,-0.38,30.79,30.30,92.0,-0.01,1.79,12.86,X40,ATH,ELECTRICAL
12,BANKINDIA,190.00,-30.43,61.0,H-MC,5.62,180469.0,661.0,111746.0,-0.64,0.37,61.92,62.52,88.0,0.01,1.31,34.58,XR,NTT,BANKS


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-16.43,39.0,H-LC,5.80,301168.0,-14674.0,127334.0,0.48,-4.65,42.28,35.67,5.0,-0.12,2.19,8.56,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.39,44.0,H-LC,13.43,267976.0,611.0,64850.0,0.13,0.23,24.20,24.48,48.0,0.01,1.95,55.33,XY24,BTT,FINANCE
31,HCLTECH,1943.91,0.22,47.0,H-LC,10.14,229204.0,-12720.0,75981.0,-0.50,-5.26,33.15,26.15,8.0,-0.17,1.67,8.31,X40,ATH,IT
80,UNITDSPR,1644.00,-13.80,48.0,H-LC,8.01,226008.0,-8136.0,56751.0,0.33,-3.47,25.11,20.77,86.0,-0.14,1.64,3.20,X40N,NTT,BREWERIES
39,INFY,2275.00,-17.80,52.0,H-LC,9.13,320906.0,8018.0,163662.0,-1.25,2.56,51.00,54.87,3.0,0.05,2.33,9.38,X40,BTT,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-3.47,59.0,H-LC,1.08,239445.0,-6706.0,85913.0,-0.22,-2.72,35.88,32.18,16.0,-0.08,1.74,32.34,X200,ATH,IT
41,ITC,452.00,-37.50,55.0,H-LC,2.90,202446.0,2308.0,18585.0,0.06,1.15,9.18,10.44,4.0,0.12,1.47,6.78,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.84,60.0,H-LC,3.94,283631.0,18205.0,38574.0,-0.45,6.86,13.60,21.40,11.0,0.47,2.06,14.44,XY25,NTT,FMCG
83,VBL,671.64,-16.43,39.0,H-LC,5.80,301168.0,-14674.0,127334.0,0.48,-4.65,42.28,35.67,5.0,-0.12,2.19,8.56,X40N,ATH,FMCG
56,RELIANCE,1533.00,-12.91,53.0,H-LC,5.86,217745.0,7379.0,21404.0,0.06,3.51,9.83,13.68,37.0,0.34,1.58,20.61,XY25,NTT,REFINERIES


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TITAGARH,1548.0,1.18,70.0,H-SC,4.96,165996.0,-26019.0,104909.0,2.27,-13.55,63.20,41.08,158.0,-0.25,1.21,40.66,XY24,NTT,ENGINEERING
53,RAJESHEXPO,518.0,1857.07,64.0,L-SC,4.97,53387.0,-83790.0,83882.0,1.79,-61.08,157.12,0.07,267.0,-1.00,0.39,32.78,OX40N,NTT,JEWELLERY
78,UJJIVANSFB,60.0,103.03,63.0,H-SC,12.51,127953.0,-14526.0,34048.0,1.65,-10.20,26.61,13.70,163.0,-0.43,0.93,51.02,OX40N,NTT,BANKS
13,BATAINDIA,2096.0,-29.01,70.0,M-SC,6.85,106041.0,-22629.0,67930.0,1.46,-17.59,64.06,35.20,219.0,-0.33,0.77,21.71,X40,NTT,FOOTWEAR
10,BAJAJHFL,152.0,-11.52,52.0,H-MC,6.13,186370.0,-14136.0,62919.0,1.44,-7.05,33.76,24.33,90.0,-0.22,1.36,7.47,X40N,BTT,FINANCE


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3791.89,61.0,M-SC,6.77,121280.0,-26088.0,77898.0,-2.06,-17.70,64.23,35.16,236.0,-0.33,0.88,23.88,XY24,NTT,AUTO
39,INFY,2275.00,-17.80,52.0,H-LC,9.13,320906.0,8018.0,163662.0,-1.25,2.56,51.00,54.87,3.0,0.05,2.33,9.38,X40,BTT,IT
28,HAPPSTMNDS,1488.71,-16.42,38.0,H-SC,10.18,90296.0,-37869.0,147896.0,-1.22,-29.55,163.79,85.85,132.0,-0.26,0.66,4.48,AR,ATH,IT
55,RELAXO,1176.00,-33.29,53.0,H-SC,1.91,90964.0,-54196.0,130124.0,-1.12,-37.34,143.05,52.31,136.0,-0.42,0.66,21.76,X40N,NTT,FOOTWEAR
63,SFL,1287.00,42.86,57.0,M-SC,8.59,188572.0,-73646.0,143466.0,-0.86,-28.09,76.08,26.63,229.0,-0.51,1.37,17.24,XY24,NTT,MISC


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,103.03,63.0,H-SC,12.51,127953.0,-14526.0,34048.0,1.65,-10.20,26.61,13.70,163.0,-0.43,0.93,51.02,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,102.77,55.0,M-SC,4.73,87377.0,-13400.0,13491.0,-0.45,-13.30,15.44,0.09,245.0,-0.99,0.64,14.79,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.61,60.0,H-SC,1.79,223794.0,-45873.0,82200.0,0.45,-17.01,36.73,13.47,138.0,-0.56,1.63,13.19,XY24,NTT,PAINTS
19,CERA,9475.0,-20.29,44.0,H-SC,2.15,145429.0,-30474.0,72496.0,0.02,-17.32,49.85,23.89,149.0,-0.42,1.06,25.60,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-19.68,54.0,H-MC,7.83,108585.0,-23250.0,65477.0,-0.11,-17.64,60.30,32.03,98.0,-0.36,0.79,21.62,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.44,52.0,H-LC,13.10,293017.0,-52939.0,176806.0,-0.52,-15.30,60.34,35.80,1.0,-0.30,2.13,3.80,X40,BTT,IT
39,INFY,2275.00,-17.80,52.0,H-LC,9.13,320906.0,8018.0,163662.0,-1.25,2.56,51.00,54.87,3.0,0.05,2.33,9.38,X40,BTT,IT
41,ITC,452.00,-37.50,55.0,H-LC,2.90,202446.0,2308.0,18585.0,0.06,1.15,9.18,10.44,4.0,0.12,1.47,6.78,X40,NTT,FMCG
83,VBL,671.64,-16.43,39.0,H-LC,5.80,301168.0,-14674.0,127334.0,0.48,-4.65,42.28,35.67,5.0,-0.12,2.19,8.56,X40N,ATH,FMCG
1,ABB,7934.00,-39.57,57.0,H-LC,11.57,252672.0,-8947.0,128155.0,0.29,-3.42,50.72,45.57,7.0,-0.07,1.84,7.51,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,44.35,65.0,M-MC,6.94,227993.0,3031.0,164634.0,0.11,1.35,72.21,74.53,192.0,0.02,1.66,34.04,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.48,53.0,H-MC,3.82,189020.0,24350.0,43626.0,0.28,14.79,23.08,41.28,89.0,0.56,1.37,23.95,X40N,ATH,AC
84,VOLTAS,1918.49,0.46,57.0,H-MC,2.84,210645.0,18903.0,77138.0,0.05,9.86,36.62,50.08,99.0,0.25,1.53,17.33,XY25,ATH,AC
24,DABUR,735.00,-3.48,56.0,H-MC,6.37,210249.0,13221.0,76404.0,0.05,6.71,36.34,45.49,102.0,0.17,1.53,20.63,XY24,BTT,FMCG
18,CAMS,5211.76,-5.16,51.0,H-SC,4.66,107867.0,5863.0,38066.0,-0.17,5.75,35.29,43.06,122.0,0.15,0.78,23.85,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.00,74.0,H-MC,5.11,116330.0,15826.0,14809.0,0.55,15.75,12.73,30.49,79.0,1.07,0.85,53.25,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.03,56.0,M-SC,1.04,101370.0,9872.0,68881.0,0.01,10.79,67.95,86.07,223.0,0.14,0.74,47.21,XR,NTT,DURABLES
36,INDIAMART,4850.92,-51.09,49.0,H-SC,12.21,136740.0,13404.0,120359.0,-0.80,10.87,88.02,108.45,119.0,0.11,0.99,35.65,AR,ATH,MISC
86,WIPRO,420.00,-11.52,51.0,M-LC,5.71,155260.0,4315.0,105142.0,-0.60,2.86,67.72,72.51,53.0,0.04,1.13,9.05,XR,NTT,IT
12,BANKINDIA,190.00,-30.43,61.0,H-MC,5.62,180469.0,661.0,111746.0,-0.64,0.37,61.92,62.52,88.0,0.01,1.31,34.58,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-171.59,64.0,M-SC,2.79,82572.0,-19638.0,113025.0,1.04,-19.21,136.88,91.37,208.0,-0.17,0.60,34.04,XY25,NTT,FINANCE
22,COFFEEDAY,80.00,-41.65,54.0,L-SC,45.64,86419.0,-27130.0,67182.0,-0.53,-23.89,77.74,35.27,268.0,-0.40,0.63,111.51,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-5.68,43.0,H-SC,13.72,87742.0,-13221.0,107756.0,0.49,-13.10,122.81,93.63,148.0,-0.12,0.64,30.69,SR,ATH,CHEMICALS
48,MASFIN,398.61,-15.54,59.0,H-SC,12.85,96090.0,-1890.0,23494.0,-0.76,-1.93,24.45,22.05,152.0,-0.08,0.70,39.67,XR,ATH,FINANCE
49,NATIONALUM,247.44,-42.00,74.0,H-MC,5.11,116330.0,15826.0,14809.0,0.55,15.75,12.73,30.49,79.0,1.07,0.85,53.25,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,54.0,L-SC,45.64,86419.0,-27130.0,67182.0,-0.53,-23.89,77.74,35.27,268.0,-0.40,0.63,111.51,XR,NTT,HOTELS
27,GREENPANEL,537.00,246.25,72.0,M-SC,4.60,154476.0,-28602.0,107577.0,1.00,-15.62,69.64,43.14,230.0,-0.27,1.12,45.31,XY24,NTT,MISC
49,NATIONALUM,247.44,-42.00,74.0,H-MC,5.11,116330.0,15826.0,14809.0,0.55,15.75,12.73,30.49,79.0,1.07,0.85,53.25,MH,ATH,METALS
8,ATULAUTO,844.00,3791.89,61.0,M-SC,6.77,121280.0,-26088.0,77898.0,-2.06,-17.70,64.23,35.16,236.0,-0.33,0.88,23.88,XY24,NTT,AUTO
60,SAMMAANCAP,326.00,-171.59,64.0,M-SC,2.79,82572.0,-19638.0,113025.0,1.04,-19.21,136.88,91.37,208.0,-0.17,0.60,34.04,XY25,NTT,FINANCE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.73
1,25,43.44
2,50,73.82


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.49
LC    34.46
MC    23.05
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.97
X40      15.23
X40N     12.98
XR       10.03
XY25      9.76
AR        8.60
OX40N     8.20
MH        1.74
X200      1.74
X5K       1.47
SR        1.28
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    27.42
H-LC    26.20
H-MC    20.09
M-SC    13.52
M-LC     7.20
M-MC     2.65
L-SC     1.55
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.35,-14.61,79.77
FMCG,11.83,-3.96,40.34
FINANCE,9.64,-16.15,61.64
BANKS,7.79,-11.85,63.35
MISC,6.04,-14.03,79.13
PAINTS,5.88,-14.35,39.95
ELECTRICAL,5.61,-6.68,65.48
AC,3.61,2.52,30.49
INSURANCE,3.40,-7.74,44.70


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2858708.0,22
AR,1315225.0,9
XR,1235681.0,13
X40,871999.0,10
X40N,626708.0,10
OX40N,521386.0,10
XY25,407659.0,7
SR,246248.0,2
X5K,115449.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3348803.0,28
H-LC,1388927.0,15
M-SC,1386807.0,17
H-MC,1189597.0,15
M-LC,387343.0,5
M-MC,314280.0,2
L-SC,248094.0,3
L-MC,60249.0,1
L-LC,38204.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1035118.0      6
M-SC       XY24       800675.0      7
H-SC       AR         785721.0      5
           XR         678347.0      6
H-LC       X40        586534.0      5
H-MC       XY24       486777.0      4
H-LC       AR         366625.0      2
M-MC       XY24       314280.0      2
H-SC       X40N       295136.0      4
           SR         246248.0      2
           OX40N      245714.0      4
H-LC       X40N       225027.0      4
H-MC       X40        217535.0      4
           XY25       186708.0      2
L-SC       XR         164212.0      2
M-SC       AR         162879.0      2
M-LC       XY24       157008.0      2
M-SC       OX40N      126313.0      4
           XR         115985.0      2
M-LC       X5K        115449.0      1
M-SC       XY25       113025.0      1
H-MC       XR         111746.0      1
           X40N       106545.0      2
M-LC       XR         105142.0      1
H-LC       X200        85913.0      1
L-SC       OX40N       83882.0      1
M-SC       X40         67930.0      1
H-MC       OX40N       65477.0      1
H-LC       XY24        64850.0      1
H-SC       MH          62519.0      1
L-MC       XR          60249.0      1
H-LC       XY25        59978.0      2
L-LC       XY25        38204.0      1
H-MC       MH          14809.0      1
M-LC       XY25         9744.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
